In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import math
from copy import deepcopy

In [2]:
class AMR():
    def __init__(self, _index, _speed):
        self.index = _index
        self.speed = _speed
        self.status = 0   # 0: idle status, 1: busy status
        self.path = []
        
    def FIFO(self, _materials):
        earliestReleaseTime = 99999999999
        earliestMaterialIndex = 9999
        
        for index, material in enumerate(_materials):
            if material.status == 0:
                if material.releaseTime < earliestReleaseTime:
                    earliestReleaseTime = material.releaseTime
                    earliestMaterialIndex = index
        
        return earliestMaterialIndex
    
    def EDD(self, _materials):
        earliestDueDate = 999999999999
        earliestMaterialIndex = 9999
        for index, material in enumerate(_materials):
            if material.status == 0: # material is unassigned
                if material.dueDate < earliestDueDate:
                    earliestDueDate = material.dueDate
                    earliestMaterialIndex = index
        return earliestMaterialIndex
    
    
    def SPT(self, _materials):
        shortestProcessTime = 999999999999
        shortestMaterialIndex = 99999
        for index, material in enumerate(_materials):
            if material.status == 0:
                if material.processTime < shortestProcessTime:
                    shortestProcessTime = material.processTime
                    shortestMaterialIndex = index
        return shortestMaterialIndex
    
    def InsertPath(self, _material):
        self.path.append(Node(_material.storage.location, _material, None, None)) # storage of _material
        self.path.append(Node(_material.workcenter.location, _material, None, None)) # workcenter of _material
        
        
    def Scheduling(self, currentTime):
        if len(self.path) > 2:
            self.path[-3].departureTime = currentTime
            self.path[-2].arrivalTime = round(self.path[-3].departureTime + self.GetTravelTime(self.path[-3].location, self.path[-2].location), 1)
            self.path[-2].departureTime = round(self.path[-2].arrivalTime + self.path[-2].material.processTime, 1)
            self.path[-1].arrivalTime = round(self.path[-2].departureTime + self.GetTravelTime(self.path[-2].location, self.path[-1].location), 1)
            self.path[-1].departureTime = round(self.path[-1].arrivalTime + self.path[-1].material.processTime, 1)
        else:
            self.path[-2].arrivalTime = currentTime
            self.path[-2].departureTime = round(self.path[-2].arrivalTime + self.path[-2].material.processTime, 1)
            self.path[-1].arrivalTime = round(self.path[-2].departureTime + self.GetTravelTime(self.path[-2].location, self.path[-1].location) , 1)
            self.path[-1].departureTime = round(self.path[-1].arrivalTime + self.path[-1].material.processTime, 1)
    
    def GetTravelTime(self, location1, location2):
        distance = math.sqrt((location1[0] - location2[0])**2 + (location1[1] - location2[1])**2)
        travelTime = distance / self.speed
        return travelTime
    


class Storage():
    def __init__(self, _index, _location):
        self.index = _index
        self.location = _location
        self.numUnAssignedMaterial = 0
        self.numUnAssignedMaterials = []
        self.interReleaseTimes = []
        self.elapsedTimes = []
        
class Workcenter():
    def __init__(self, _index, _location):
        self.index = _index
        self.location = _location
        
class Material():
    def __init__(self, _index, _releaseTime, _processTime, _dueDate, _storage, _workcenter):
        self.index = _index
        self.releaseTime = _releaseTime
        self.processTime = _processTime
        self.dueDate = _dueDate
        self.storage = _storage
        self.workcenter = _workcenter
        self.status = 0  # 0: unassigned, 1: assigned, 2: finished
        
class Node():
    def __init__(self, _location, _material, _arrivalTime, _departureTime):
        self.location = _location
        self.material = _material
        self.arrivalTime = _arrivalTime
        self.departureTime = _departureTime

class Chromosome():
    def __init__(self):
        self.genes = []
        self.fitnessValue = 0
        self.amr = None

In [3]:
def GetFitnessValue(amr, materials, chromosome):
    # initialize AMR
    amr.status = 0
    amr.path = []
    
    # initialize materials
    for material in materials:
        material.status = 0
        
    currentTime = 0
    geneIndex = 0
    
    while True:
        StatusUpdate(currentTime)
        if AllDone() == True:
            break

        for amr in amrs:
            if IsUnAssigned() == False: # there are no unassigned materials
                break
            if amr.status == 0:

                
                if chromosome.genes[geneIndex] == 0:
                    materialIndex = amr.FIFO(materials)
                elif chromosome.genes[geneIndex] == 1:
                    materialIndex = amr.EDD(materials)
                elif chromosome.genes[geneIndex] == 2:
                    materialIndex = amr.SPT(materials)
                
                
                amr.InsertPath(materials[materialIndex])
                amr.Scheduling(currentTime)
                materials[materialIndex].status = 1   # assigned
                amr.status = 1 # busy
                geneIndex += 1

        currentTime = round(currentTime + 0.1, 1)
        
        
    chromosome.amr = deepcopy(amr)
    chromosome.fitnessValue = amr.path[-1].departureTime # makespan

In [4]:
def StatusUpdate(currentTime):
    for amr in amrs:
        if len(amr.path) > 0:
            if round(amr.path[-1].arrivalTime + amr.path[-1].material.processTime, 1) <= currentTime:
                amr.status = 0
                amr.path[-1].material.status = 2
                
def AllDone():
    flag = True
    for material in materials:
        if material.status != 2:
            flag = False
            break
    return flag

def IsUnAssigned():
    flag = False
    for material in materials:
        if material.status == 0: # unAssigned
            flag = True
            break
    return flag

### parameters

In [5]:
numStorages = 3
numWorkcenters = 3
mapWidth = 50
mapHeight = 50

import random
random.seed(1234)

storages = []
storage = Storage(0, (1, 10))
storages.append(storage)
storage = Storage(1, (3, 20))
storages.append(storage)
storage = Storage(2, (4, 15))
storages.append(storage)


workcenters = []
workcenter = Workcenter(0, (24, 24))
workcenters.append(workcenter)
workcenter = Workcenter(1, (20, 0))
workcenters.append(workcenter)
workcenter = Workcenter(2, (15, 15))
workcenters.append(workcenter)


numMaterials = 30
maxReleaseTime = 1000
maxProcessTime = 10
minDueDate = 10
maxDueDate = 15


materials = []
for i in range(numMaterials):
    releaseTime = random.randint(0, maxReleaseTime)
    processTime = random.randint(0, maxProcessTime)
    dueDate = random.randint(minDueDate, maxDueDate)
    dueDate = releaseTime + dueDate
    storageIndex = random.randint(0, numStorages - 1)
    workcenterIndex = random.randint(0, numWorkcenters - 1)
    material = Material(i, releaseTime, processTime, dueDate, storages[storageIndex], workcenters[workcenterIndex])
    materials.append(material)


In [6]:
numAMRs = 1

amrs = []
for i in range(numAMRs):
    amr = AMR(i, 1)
    amrs.append(amr)

In [11]:
# generate new materials
materials = []
for i in range(numMaterials):
    releaseTime = random.randint(0, maxReleaseTime)
    processTime = random.randint(0, maxProcessTime)
    dueDate = random.randint(minDueDate, maxDueDate)
    dueDate = releaseTime + dueDate
    storageIndex = random.randint(0, numStorages - 1)
    workcenterIndex = random.randint(0, numWorkcenters - 1)
    material = Material(i, releaseTime, processTime, dueDate, storages[storageIndex], workcenters[workcenterIndex])
    materials.append(material)

initialPopulationSize = 20

chromosomes = []

for i in range(initialPopulationSize):
    
    chromosome = Chromosome()
    
    for j in range(numMaterials):
        # randomly select dispatching rule
        dispatchingRuleIndex = random.randint(0, 2)
        chromosome.genes.append(dispatchingRuleIndex)
    chromosomes.append(chromosome)
    
# generate path and calculate the fitness value
for chromosome in chromosomes:
    GetFitnessValue(amrs[0], materials, chromosome)

# Generation
maxGeneration = 5
for i in range(maxGeneration):
    # sort the chromosomes by fitness values
    chromosomes = sorted(chromosomes, key=lambda chromosome: chromosome.fitnessValue)
    
    # choose the best two chromosomes as parents
    chromosome_0 = deepcopy(chromosomes[0])
    chromosome_1 = deepcopy(chromosomes[1])
    
    # crossover
    # randomly swap 1 gene
    swapIndex = random.randint(0, len(chromosome_0.genes) - 1)
    
    tempGene = chromosome_0.genes[swapIndex]
    chromosome_0.genes[swapIndex] = chromosome_1.genes[swapIndex]
    chromosome_1.genes[swapIndex] = tempGene
    
    # generate path and calculate the fitness value
    GetFitnessValue(amrs[0], materials, chromosome_0)
    GetFitnessValue(amrs[0], materials, chromosome_1)
    
    # add to population
    chromosomes.append(chromosome_0)
    chromosomes.append(chromosome_1)
    
    
    # mutation
    mutatedChromosomeIndex = random.randint(0, len(chromosomes) - 1)
    mutatedChromosome = deepcopy(chromosomes[mutatedChromosomeIndex])
    mutatedGeneIndex_0 = random.randint(0, len(mutatedChromosome.genes) - 1)
    mutatedGeneIndex_1 = random.randint(0, len(mutatedChromosome.genes) - 1)
    
    
    tempGene = mutatedChromosome.genes[mutatedGeneIndex_0]
    mutatedChromosome.genes[mutatedGeneIndex_0] = mutatedChromosome.genes[mutatedGeneIndex_1]
    mutatedChromosome.genes[mutatedGeneIndex_1] = tempGene
    
    # generate path and calculate the fitness value
    GetFitnessValue(amrs[0], materials, mutatedChromosome)
    
    # add to the population
    chromosomes.append(mutatedChromosome)
    
    chromosomes = sorted(chromosomes, key=lambda chromosome: chromosome.fitnessValue)
    
    # remove the worst chromosomes
    del chromosomes[initialPopulationSize: len(chromosomes)]

    
# find the best 5 chromosomes
bestChromosomes = chromosomes[0:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)


'''
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 3, random_state = 0)
tree.fit(X_train, y_train) # learn our model
print("train accuracy: {:.3f}".format(tree.score(X_train, y_train)))
print("test accuracy: {:.3f}".format(tree.score(X_test, y_test)))
'''

# Set data types for categorical labels
from tensorflow.keras import utils
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)


# Define a classifier network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
hl = 1 # number of hidden layer nodes

model = Sequential()
model.add(Dense(hl, input_dim = X_train.shape[1], activation='relu'))
model.add(Dense(hl, input_dim = hl, activation='relu'))
model.add(Dense(y_train.shape[1], input_dim = hl, activation='softmax'))

# Train the model
# hyper parameters for optimizer
from tensorflow.keras import optimizers

opt = optimizers.Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# Train the model over 50 epochs using 10-observation batches
num_epochs = 50
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=10, validation_data=(X_test, y_test))


# Use the trained model

# generate new materials (to make a variety of training instances)
materials = []
for i in range(numMaterials):
    releaseTime = random.randint(0, maxReleaseTime)
    processTime = random.randint(0, maxProcessTime)
    dueDate = random.randint(minDueDate, maxDueDate)
    dueDate = releaseTime + dueDate
    storageIndex = random.randint(0, len(storages) - 1)
    workcenterIndex = random.randint(0, len(workcenters) - 1)
    material = Material(i, releaseTime, processTime, dueDate, storages[storageIndex], workcenters[workcenterIndex])
    materials.append(material)


result_FIFO = 0
result_EDD = 0
result_SPT = 0
result_ML = 0

max_case = 4
for case in range(max_case):
    currentTime = 0
    
    # initialize AMR
    for amr in amrs:
        amr.status = 0
        amr.path = []
        
    # initialize the statsu of materials
    for material in materials:
        material.status = 0

    while True:
        StatusUpdate(currentTime)
        if AllDone() == True:
            break
        
        for amr in amrs:
            
            if IsUnAssigned() == False: # there are no unassigned materials
                break
            
            if amr.status == 0:
                
                # Dispatching rule
                if case == 0:
                    materialIndex = amr.FIFO(materials)
                elif case == 1:
                    materialIndex = amr.EDD(materials)
                elif case == 2:
                    materialIndex = amr.SPT(materials)
                elif case == 3:
                    
                    # initialize features
                    feature1 = []
                    for storage in storages:
                        storage.numUnAssignedMaterial = 0
                        storage.numUnAssignedMaterials = []
                        storage.interReleaseTimes = []
                        storage.elapsedTimes = []
                    
                    
                    departureTime_targetAMR = currentTime
                    
                    # Feature 1: Distance between target AMR and storage where the unassigned material is held
                    earliestReleaseTime = 999999
                    earliestReleaselocation = (0, 0)
                    for material in materials:
                        if material.releaseTime < earliestReleaseTime:
                            earliestReleaseTime = material.releaseTime
                            earliestReleaselocation = material.storage.location
                    
                    if len(amr.path) == 0:
                        distance = 0.0
                    else:
                        distance = math.sqrt((amr.path[-1].location[0] - earliestReleaselocation[0])**2 + (amr.path[-1].location[1] - earliestReleaselocation[1])**2)
                
                    feature1.append(distance)




                    # Feature 2: The number of currently unassigned materials at each storage
                    for material in materials:
                        if material.status == 0:
                            material.storage.numUnAssignedMaterial += 1
                            
                    for storage in storages:
                        storage.numUnAssignedMaterials.append(storage.numUnAssignedMaterial)
                    

                    # Feature 3: Average inter-release time of materials at each storage
                    for storage in storages:
                        releaseTimes = []   
                        
                        for material in materials:
                            if material.storage.index == storage.index and material.releaseTime <= currentTime:
                                releaseTimes.append(material.releaseTime)
                                
                        releaseTimes.sort()
                        interReleaseTimes = []
                        for index in range(len(releaseTimes) - 1):
                            interReleaseTimes.append(releaseTimes[index + 1] - releaseTimes[index])
                        
                        if interReleaseTimes:
                            interReleaseTimes = np.array(interReleaseTimes)
                            storage.interReleaseTimes.append(np.mean(interReleaseTimes))
                        else:
                            storage.interReleaseTimes.append(0)
                    
                        # Feature 4: Elapsed time since the last released material at each storage                                            
                        elapsedTime = 0
                        for releaseTime in releaseTimes:
                            if releaseTime <= departureTime_targetAMR:
                                elapsedTime = departureTime_targetAMR - releaseTime
                            else:
                                break
                                
                        storage.elapsedTimes.append(elapsedTime)


                    # make pandas dataframe
                    X_new = pd.DataFrame({"feature1": feature1, 'feature2_0': storages[0].numUnAssignedMaterials, 'feature2_1': storages[1].numUnAssignedMaterials, 'feature2_2': storages[2].numUnAssignedMaterials,
                     'feature3_0': storages[0].interReleaseTimes, 'feature3_1': storages[1].interReleaseTimes, 'feature3_2': storages[2].interReleaseTimes, 'feature4_0': storages[0].elapsedTimes, 
                     'feature4_1': storages[1].elapsedTimes, 'feature4_2': storages[2].elapsedTimes})


                    class_probabilities = model.predict(X_new)
                    predicionts = np.argmax(class_probabilities, axis=1)
                    
                    if predicionts[0] == 0:
                        materialIndex = amr.FIFO(materials)
                    elif predicionts[0] == 1:
                        materialIndex = amr.EDD(materials)
                    elif predicionts[0] == 2:
                        materialIndex = amr.SPT(materials)

                
                amr.InsertPath(materials[materialIndex])
                amr.Scheduling(currentTime)
                materials[materialIndex].status = 1
                amr.status = 1
                
        currentTime = round(currentTime + 0.1, 1)
        
    # performance
    makeSpan = 0
    totalTardiness = 0
    for amr in amrs:
        if len(amr.path) > 0:
            # makespan
            if makeSpan < amr.path[-1].departureTime:
                makeSpan = amr.path[-1].departureTime
        
        for index, node in enumerate(amr.path):
            if index % 2 == 1:
                totalTardiness += max(node.arrivalTime + node.material.processTime - node.material.dueDate, 0)
                
    if case == 0:
        result_FIFO = totalTardiness
    elif case == 1:
        result_EDD = totalTardiness
    elif case == 2:
        result_SPT = totalTardiness
    elif case == 3:
        result_ML = totalTardiness
        
print('totalTardiness (FIFO): ' + str(result_FIFO))
print('totalTardiness (EDD): ' + str(result_EDD))
print('totalTardiness (SPT): ' + str(result_SPT))
print('totalTardiness (ML): ' + str(result_ML))

In [ ]:
# csv file export
AMRs_indices = []
AMRs_currentLocations_x = []
AMRs_currentLocations_y = []
AMRs_currentTimes = []
for amr in amrs:
    indices = []
    currentLocations_x = []
    currentLocations_y = []
    currentTimes = []

    currentTime = 0
    while True:
        for i in range(len(amr.path) - 1):
            if round(amr.path[i].arrivalTime, 1) <= currentTime <= round(amr.path[i].departureTime, 1):
                indices.append(amr.index)
                currentTimes.append(currentTime)
                currentLocations_x.append(amr.path[i].location[0])
                currentLocations_y.append(amr.path[i].location[1])
                

            elif round(amr.path[i].departureTime, 1) < currentTime < round(amr.path[i + 1].arrivalTime, 1):
                indices.append(amr.index)
                currentTimes.append(currentTime)
                travelTimeBtwNextAndPrev = amr.path[i + 1].arrivalTime - amr.path[i].departureTime
                travelTimeBtwCurrentAndPrev = currentTime - amr.path[i].departureTime
                ratio = travelTimeBtwCurrentAndPrev / travelTimeBtwNextAndPrev
                
                directionVector = (amr.path[i + 1].location[0] - amr.path[i].location[0], amr.path[i + 1].location[1] - amr.path[i].location[1])
                
                currentLocations_x.append(amr.path[i].location[0] + (directionVector[0] * ratio))
                currentLocations_y.append(amr.path[i].location[1] + (directionVector[1] * ratio))
        
        currentTime = round(currentTime + 0.1, 1)
        
        if currentTime == round(amr.path[-1].arrivalTime, 1):
            indices.append(amr.index)
            currentTimes.append(currentTime)
            currentLocations_x.append(amr.path[-1].location[0])
            currentLocations_y.append(amr.path[-1].location[1])
            break
    
    AMRs_indices.extend(indices)
    AMRs_currentLocations_x.extend(currentLocations_x)
    AMRs_currentLocations_y.extend(currentLocations_y)
    AMRs_currentTimes.extend(currentTimes)


# make pandas dataframe
result = pd.DataFrame({"amrIndex": AMRs_indices, 'location_x': AMRs_currentLocations_x, 'location_y': AMRs_currentLocations_y, 'currentTime': AMRs_currentTimes})
result.to_csv('mycsvfile.csv',index=False)
print('end')
